In [7]:
import polars as pl
import os
from pathlib import Path
from datetime import datetime
from datetime import date

path = "/home/ubuntu/bvic/ibmb_script/ccw/flagged/daily_ccw_flag.parquet"

path = Path(path)

if os.path.exists(path):
    df = pl.scan_parquet(path)
    # print(df.head(10))
else:
    print("File not found:", path)
    # maybe create empty DataFrame instead
    # df = pl.DataFrame()


In [8]:
df.head().collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag,temp_date_col
datetime[μs],str,str,str,str,str,str,i8,f64,str,str,i8,str,i8,i8


In [3]:
df = df.filter(pl.col("transaction_state") == 1)

In [4]:
df = df.filter(pl.col("channel_code") == "MB")

In [38]:
df = df.filter(pl.col("transaction_amount") > 100000)

In [39]:
df

In [7]:
df_rolling = (
            df.rolling(
                index_column="transaction_date",
                period="10m",
                group_by="from_account_number"
            ).agg(pl.len()).rename({"len": "rolling_count"})
        )
df_with_rolling = (
    df.join(
        df_rolling,
        on=["from_account_number", "transaction_date"],
        how="left")
)
burst_end = (
    df_rolling.filter(
        pl.col("rolling_count") >= 3).select(["from_account_number", "transaction_date"])
)
# burst_end: ensure "t_start" column exists
burst_end = (
    burst_end.with_columns(
        (pl.col("transaction_date") - pl.duration(minutes=10)).alias("t_start"))
)
df_with_rolling_with_burst = (df_with_rolling.join(
        burst_end, on=["from_account_number"], how="inner"))

flag_only = (
    df_with_rolling_with_burst.filter(
        (pl.col("transaction_date") >= pl.col("t_start")) &
        (pl.col("transaction_date") <= pl.col("transaction_date_right"))).unique(subset=["no_referensi"])
)
flag_only = (
    flag_only.with_columns(
        pl.lit(1).cast(pl.Int8).alias("flag"))
)

In [6]:
flag_only

In [7]:
df.select(pl.len()).collect()

len
u32
389


In [4]:
shape = (df.select(pl.len()).collect()[0, 0], len(df.collect_schema().names()))
print(shape)

(286, 14)


In [4]:
df.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag
datetime[μs],str,str,str,str,str,str,i8,f64,str,str,i8,str,i8
2025-08-12 18:51:55,"""9771140281""","""PRIA FAUZAN""",null,null,"""ATM BSI""","""CCW""",0,300000.0,"""IDR""","""MB""",1,"""MB4996896777""",0
2025-08-13 07:38:38,"""8228381222""","""DENNIS TRIYANDA PUTRA""",null,null,"""ATM BRI""","""CCW""",0,200000.0,"""IDR""","""MB""",1,"""MB4319893187""",0
2025-08-13 10:52:45,"""5210000013""","""MITA RISMAWATI""",null,null,"""ATM BRI""","""CCW""",0,200000.0,"""IDR""","""MB""",1,"""MB6181890812""",0
2025-08-14 13:55:14,"""5210000013""","""MITA RISMAWATI""",null,null,"""ATM BRI""","""CCW""",0,300000.0,"""IDR""","""MB""",1,"""MB0641877261""",0
2025-08-14 18:56:06,"""0661008369""","""VINI VIRGINA VALENTIN SAMPOW""",null,null,"""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB0596873895""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-11-29 23:45:54,"""0481083052""","""RD REYHAN TRIWIBOWO""",null,null,"""ATM BRI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB7238448270""",0
2025-11-30 00:30:09,"""6089115094""","""PUTRI BR SIAHAAN""",null,null,"""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB1491448090""",0
2025-11-30 12:11:15,"""0261006561""","""M.NURFAZRI""",null,null,"""ATM BSI""","""CCW""",0,200000.0,"""IDR""","""MB""",1,"""MB4734442609""",0


In [ ]:
df.collect().filter(pl.col("no_refrensi")).is_duplicated()

In [ ]:
df.collect().filter(df.collect().is_duplicated())

In [ ]:
rows = 0

In [ ]:
if not rows:
    print("a")
else:
    print("b")

In [ ]:
df.sort("transaction_date", descending=True).collect()

In [ ]:
df.filter(
    pl.col("flag") == 1
).collect()

In [ ]:
if "flag" in df.collect().columns:
    print("There is flag")
else:
    print("None found")

In [ ]:
df_pl.head()

In [ ]:
date_string = "2025-05-04 12:31:02"
format_string = "%Y-%m-%d %H:%M:%S"
datetime_object = datetime.strptime(date_string, format_string)
today = date.today()
dt_t = date(2025, 5, 4)

print(datetime_object)
print(type(datetime_object))
print(today)
print(dt_t)

In [ ]:
df_pl = df_pl.with_columns(
    pl.col("transaction_date").str.strptime(pl.Datetime, format=format_string))
df_pl = df_pl.with_columns(
    pl.col("transaction_date").dt.date().alias("date_only"))
df_pl.filter(
    pl.col("date_only") >= dt_t)

In [ ]:
format_datetime = "%Y-%m-%d %H:%M:%S"
n_days = 2

from datetime import timedelta



In [ ]:
cutoff_date = datetime.now().date() - timedelta(days=n_days)
df_pl = df_pl.with_columns(
    pl.col("transaction_date").str.strptime(
        pl.Datetime, format=format_datetime)
)
df_pl = df_pl.with_columns(
    pl.col("transaction_date").dt.date().alias("date_only"))
df_pl = df_pl.filter(
    pl.col("date_only") >= cutoff_date)

In [ ]:
df_pl

In [ ]:
# # df_pl = df_pl.with_columns(
# #     pl.col("transaction_date").str.strptime(pl.Datetime, format=format_string))
# df_pl = df_pl.with_columns(
#     pl.col("transaction_date").dt.date().alias("date_only"))
# df_pl.filter(
#     pl.col("date_only") == today)

In [ ]:
import pandas as pd

df = pd.read_parquet("/home/ubuntu/bvic/data/ccw/ccw_historical/data/ccw_historical_data.parquet")
df = df.sort_values(["from_account_number", "transaction_date"])
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

prev_result = (
            df.set_index("transaction_date")
            .groupby("from_account_number")
            .rolling(window=pd.Timedelta(minutes=10))
            .count()
            .reset_index()
        )

rolling_count = (
    df.groupby("from_account_number") 
      .apply(lambda g: 
             g.set_index("transaction_date")["from_account_number"].rolling("10min").count()
      )
      .reset_index(name="rolling_count")
)
df = df.merge(
    rolling_count,
    on=["from_account_number", "transaction_date"],
    how="left"
)
burst_end = (
    df[df["rolling_count"] >= 3][["from_account_number", "transaction_date"]]
)
# df["flag"] = False

# for _, row in burst_end.iterrows():
#     acc = row["from_account_number"]
#     t_end = row["transaction_date"]
#     t_start = t_end - pd.Timedelta(minutes=10)
#     mask = (
#         (df["from_account_number"] == acc) &
#         (df["transaction_date"] >= t_start) &
#         (df["transaction_date"] <= t_end)
#     )
#     df.loc[mask, "flag"] = True

In [ ]:
df

In [ ]:
burst_end

In [ ]:
df[df["flag"]].head(10)

In [ ]:
import polars as pl

# Load
df = pl.read_parquet("/home/ubuntu/bvic/data/ccw/ccw_historical/data/ccw_historical_data.parquet")

df = df.with_columns([
    pl.col("transaction_date").str.to_datetime()
]).sort(["transaction_date", "from_account_number"])

df_rolling = df.rolling(
    index_column="transaction_date",
    group_by="from_account_number",
    period="10m"
).agg(pl.len()).rename({"len": "rolling_count"})

In [ ]:
df_with_rolling = df.join(
    df_rolling,
    on=["from_account_number", "transaction_date"],
    how="left"
)

In [ ]:
burst_end = df_rolling.filter(
    pl.col("rolling_count") >= 3).select(["from_account_number", "transaction_date"]
)


In [ ]:
burst_end

In [ ]:
# burst_end: ensure "t_start" column exists
burst_end = burst_end.with_columns(
    (pl.col("transaction_date") - pl.duration(minutes=10)).alias("t_start")
)

In [ ]:
burst_end

In [ ]:
df_with_rolling_with_burst = df_with_rolling.join(
    burst_end,
    on=["from_account_number"],
    how="inner"
)

In [ ]:
df_with_rolling_with_burst

In [ ]:
df_fl_2 = df_with_rolling_with_burst.filter(
    (pl.col("transaction_date") >= pl.col("t_start")) &
    (pl.col("transaction_date") <= pl.col("transaction_date_right"))
).unique(subset=["no_referensi"])

In [ ]:
df_fl_2 = df_fl_2.with_columns(
    pl.lit(True).alias("flag")
)

In [1]:
import polars as pl

path_hist = "/home/ubuntu/bvic/ibmb_script/ccw/data/ccw_historical_data.parquet"

df_hist = pl.scan_parquet(path_hist)

In [2]:
daily_path = "/home/ubuntu/bvic/data/new_ibmb_data/*.csv"

df_daily = pl.scan_csv(daily_path, separator="|")

In [3]:
ccw = df_daily.filter(
        (pl.col("transaction_status").cast(pl.Int64).is_in([0])) &
        (pl.col("transaction_state").cast(pl.Int64).is_in([1])) &
        (pl.col("transaction_category").is_in(["CCW"]))
            )

In [4]:
ccw.collect().shape

(389, 13)

In [6]:
df_hist.collect().shape

(57, 14)

In [7]:
delta_new = ccw.join(
                df_hist.lazy(),
                on="no_referensi",
                how="anti"
            )  # Get records from daily not in history

delta_old = df_hist.lazy().join(
    ccw,
    on="no_referensi",
    how="anti"
)

In [16]:
delta_old.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag
str,str,str,str,str,str,str,i64,f64,str,str,i64,str,i32


In [12]:
delta_new = delta_new.with_columns(
    pl.lit(0).cast(pl.Int16).alias('flag')
)

delta_new = delta_new.with_columns(
    pl.col("transaction_amount").cast(pl.Float64)
)

In [24]:
delta_old = delta_old.with_columns(
    pl.col("flag").cast(pl.Int16)
)

In [19]:
delta_new = delta_new.with_columns(
    pl.col("transaction_state").cast(pl.Int64)
)

In [20]:
delta_new.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag
str,str,str,str,str,str,str,i64,f64,str,str,i64,str,i16
"""2025-05-01 14:19:54""","""0811057550""","""JOEL EKA MARTONO KRISSATA""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB6547473504""",0
"""2025-05-02 18:27:59""","""0811028843""","""ACHMAD SOFYANADI""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB7063451388""",0
"""2025-05-02 18:49:58""","""0721023338""","""JOEL EKA MARTONO KRISSATA""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB3465450752""",0
"""2025-05-02 22:08:09""","""3534619632""","""FAISAL HAMZAH""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB5268448158""",0
"""2025-05-04 12:31:02""","""0888047824""","""EDWIN ZAINUDIN S.KOM""","""null""","""null""","""ATM BRI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB4812429820""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-11-01 08:35:53""","""8228381222""","""DENNIS TRIYANDA PUTRA""","""null""","""null""","""ATM BSI""","""CCW""",0,200000.0,"""IDR""","""MB""",1,"""MB8583896768""",0
"""2025-11-02 09:26:50""","""0691022013""","""BOUTI HERDA NINGSIH""","""null""","""null""","""ATM BRI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB9270875366""",0
"""2025-11-02 10:38:05""","""7111013211""","""PRIYAN BAGUS JOKO S AMD KEP""","""null""","""null""","""ATM BRI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB4972874241""",0


In [25]:
unique_records = pl.concat([delta_new, delta_old])

In [ ]:
unique_records
unique_records

In [27]:
unique_records.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag
str,str,str,str,str,str,str,i64,f64,str,str,i64,str,i16
"""2025-05-01 14:19:54""","""0811057550""","""JOEL EKA MARTONO KRISSATA""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB6547473504""",0
"""2025-05-02 18:27:59""","""0811028843""","""ACHMAD SOFYANADI""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB7063451388""",0
"""2025-05-02 18:49:58""","""0721023338""","""JOEL EKA MARTONO KRISSATA""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB3465450752""",0
"""2025-05-02 22:08:09""","""3534619632""","""FAISAL HAMZAH""","""null""","""null""","""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB5268448158""",0
"""2025-05-04 12:31:02""","""0888047824""","""EDWIN ZAINUDIN S.KOM""","""null""","""null""","""ATM BRI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB4812429820""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""2025-11-01 08:35:53""","""8228381222""","""DENNIS TRIYANDA PUTRA""","""null""","""null""","""ATM BSI""","""CCW""",0,200000.0,"""IDR""","""MB""",1,"""MB8583896768""",0
"""2025-11-02 09:26:50""","""0691022013""","""BOUTI HERDA NINGSIH""","""null""","""null""","""ATM BRI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB9270875366""",0
"""2025-11-02 10:38:05""","""7111013211""","""PRIYAN BAGUS JOKO S AMD KEP""","""null""","""null""","""ATM BRI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB4972874241""",0


In [15]:
import polars as pl
import os
from pathlib import Path
from datetime import datetime
from datetime import date

path = "/home/ubuntu/bvic/ibmb_script/ccw/flagged/daily_ccw_flag.parquet"

path = Path(path)

if os.path.exists(path):
    df = pl.scan_parquet(path)
    # print(df.head(10))
else:
    print("File not found:", path)
    # maybe create empty DataFrame instead
    # df = pl.DataFrame()


In [16]:
df.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag,temp_date_col,rolling_count,transaction_date_right,t_start
datetime[μs],str,str,str,str,str,str,i8,f64,str,str,i8,str,i32,date,u32,datetime[μs],datetime[μs]
2025-09-04 17:30:58,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB0513945487""",1,2025-09-04,2,2025-09-04 17:32:02,2025-09-04 17:22:02
2025-09-02 15:59:11,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB5428977432""",1,2025-09-02,3,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-07-24 18:43:01,"""0221967256""","""YULISWANDI""",null,null,"""ATM BSI""","""CCW""",0,1.2e6,"""IDR""","""MB""",1,"""MB8738649032""",1,2025-07-24,2,2025-07-24 18:44:30,2025-07-24 18:34:30
2025-09-02 16:00:31,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB0717977409""",1,2025-09-02,4,2025-09-02 16:00:31,2025-09-02 15:50:31
2025-09-02 15:56:43,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1.5e6,"""IDR""","""MB""",1,"""MB4840977471""",1,2025-09-02,1,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-07-24 18:41:43,"""0221967256""","""YULISWANDI""",null,null,"""ATM BSI""","""CCW""",0,1.3e6,"""IDR""","""MB""",1,"""MB1567649061""",1,2025-07-24,1,2025-07-24 18:44:30,2025-07-24 18:34:30
2025-09-02 15:57:47,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB7866977454""",1,2025-09-02,2,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-07-24 18:44:30,"""0221967256""","""YULISWANDI""",null,null,"""ATM BSI""","""CCW""",0,100000.0,"""IDR""","""MB""",1,"""MB6734649011""",1,2025-07-24,3,2025-07-24 18:44:30,2025-07-24 18:34:30
2025-09-04 17:32:02,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB6750945479""",1,2025-09-04,3,2025-09-04 17:32:02,2025-09-04 17:22:02


In [5]:
import polars as pl
import os
from pathlib import Path
from datetime import datetime
from datetime import date

path = "/home/ubuntu/bvic/ibmb_script/ccw/flagged/daily_ccw_flag.parquet"

path = Path(path)

if os.path.exists(path):
    df = pl.scan_parquet(path)
    # print(df.head(10))
else:
    print("File not found:", path)
    # maybe create empty DataFrame instead
    # df = pl.DataFrame()


In [6]:
df.collect()

transaction_date,from_account_number,from_account_owner_name,to_account_number,to_account_owner_name,to_bank_name,transaction_category,transaction_status,transaction_amount,transaction_currency,channel_code,transaction_state,no_referensi,flag,temp_date_col,rolling_count,transaction_date_right,t_start
datetime[μs],str,str,str,str,str,str,i8,f64,str,str,i8,str,i32,date,u32,datetime[μs],datetime[μs]
2025-09-02 16:00:31,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB0717977409""",1,2025-09-02,4,2025-09-02 16:00:31,2025-09-02 15:50:31
2025-09-04 17:30:06,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1.5e6,"""IDR""","""MB""",1,"""MB4434945496""",1,2025-09-04,1,2025-09-04 17:32:02,2025-09-04 17:22:02
2025-09-02 15:56:43,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1.5e6,"""IDR""","""MB""",1,"""MB4840977471""",1,2025-09-02,1,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-09-04 17:32:02,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,500000.0,"""IDR""","""MB""",1,"""MB6750945479""",1,2025-09-04,3,2025-09-04 17:32:02,2025-09-04 17:22:02
2025-09-02 15:57:47,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB7866977454""",1,2025-09-02,2,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-09-02 15:59:11,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB5428977432""",1,2025-09-02,3,2025-09-02 15:59:11,2025-09-02 15:49:11
2025-09-04 17:30:58,"""0971024551""","""FAIZAL HAMZAH""",null,null,"""ATM BSI""","""CCW""",0,1e6,"""IDR""","""MB""",1,"""MB0513945487""",1,2025-09-04,2,2025-09-04 17:32:02,2025-09-04 17:22:02


In [2]:
import polars as pl

# Define the column names and types
columns = ["column1", "column2", "column3"]
types = [pl.Int32, pl.Utf8, pl.Float64]

# Create the blank DataFrame
df = pl.DataFrame({col: pl.Series(col, dtype=col_type) for col, col_type in zip(columns, types)})

# Show the DataFrame
print(df)


shape: (0, 3)
┌─────────┬─────────┬─────────┐
│ column1 ┆ column2 ┆ column3 │
│ ---     ┆ ---     ┆ ---     │
│ i32     ┆ str     ┆ f64     │
╞═════════╪═════════╪═════════╡
└─────────┴─────────┴─────────┘


In [3]:
df = df.with_columns(
    pl.lit(None, dtype=pl.Int8).alias("flag")
)

In [4]:
df

column1,column2,column3,flag
i32,str,f64,i8
